In [1]:
# !pip3 install bbknn
print('Install bbknn')

Install bbknn


In [2]:
import numpy as np
import bbknn
# import pandas as pd
import matplotlib.pyplot as pl
from matplotlib import rcParams
import scanpy as sc
sc.settings.verbosity = 3  # verbosity: errors (0), warnings (1), info (2), hints (3)
sc.logging.print_versions()

scanpy==1.4 anndata==0.6.18 numpy==1.16.2 scipy==1.1.0 pandas==0.23.4 scikit-learn==0.20.3 statsmodels==0.9.0 python-igraph==0.7.1 louvain==0.6.1 


In [3]:
# Create folder to save the results 
import os
dirname = os.getcwd()
print(dirname)
data_dir = os.path.join(dirname, './data/dataset1_uc3/')
if not os.path.exists('./results/'): os.makedirs('./results/')
save_dir = os.path.join(dirname, './results/')
def save_images(basename):
    if not os.path.exists('./figures/'): os.makedirs('./figures/')
    outname = './figures/' + basename + '.png'
    pl.savefig(outname, dpi=150)
    pl.close()

/data_n3/home/hoatran/python-workspace/dca_scGen


In [4]:
# read data from read count text table, data from R: genes x cells
# adata = sc.read_text(os.path.join(data_dir,'dataset1_sm_uc3.txt'),delimiter='\t',first_column_names=True,dtype='float64')
# print(adata)
# print(len(adata.obs_names))
# print(adata.var_names)
# Read sample into a pandas series
# sample_adata = pd.read_csv(os.path.join(data_dir,'sample_sm_uc3.txt'),header=0, index_col=0, sep='\t')
# print(sample_adata.values.shape)
# print(sample_adata.keys())

# Data have format genes x cells
# But input for scanpy package require the format of cells x genes, samples x features
# We need to copyt data into new adata object with inverse raw data 
# adata1 = sc.AnnData(adata.X.T)
# adata1.var_names = adata.obs_names
# adata1.obs_names = adata.var_names
# adata1
# Save label information into adata object, same idea as Seurat 
# adata1.obs['cell_type'] = sample_adata.loc[adata1.obs_names,['celltype']]
# adata1.obs['batch'] = sample_adata.loc[adata1.obs_names,['batch']]
# print(len(adata1.obs['cell_type']))
# print(len(adata1.obs['batch']))
# Save output into h5ad, easy to access 
# adata1.write_h5ad(os.path.join(data_dir,'dataset1_uc3.h5ad'))

adata1 = sc.read_h5ad(os.path.join(data_dir,'dataset1_uc3.h5ad'))
sc.pp.filter_cells(adata1, min_genes=300)
sc.pp.filter_genes(adata1, min_cells=5)
# adata1.write_h5ad(os.path.join(data_dir,'filtered_dataset1_uc3.h5ad'))
print('Read and filter data')
# adata = sc.read_h5ad(os.path.join(data_dir,'filtered_dataset1_uc3.h5ad'))
adata1

filtered out 11 cells that have less than 300 genes expressed
filtered out 9999 genes that are detected in less than 5 cells
Read and filter data


AnnData object with n_obs × n_vars = 565 × 16594 
    obs: 'cell_type', 'batch', 'n_genes'
    var: 'n_cells'

In [5]:
sc.pp.log1p(adata1)
sc.pp.scale(adata1)
sc.tl.pca(adata1, svd_solver='arpack')
# sc.pp.neighbors(adata1,n_neighbors=10, n_pcs=20)

computing neighbors
    using 'X_pca' with n_pcs = 20
    finished (0:00:03.79) --> added to `.uns['neighbors']`
    'distances', distances for each pair of neighbors
    'connectivities', weighted adjacency matrix


In [6]:
# BBKNN
# input: batch vector in ann data

import time
from datetime import timedelta
t1 = time.time()
adata_bbknn = bbknn.bbknn(adata1, save_knn=True, copy=True, neighbors_within_batch=5,
                          approx=False,trim=50)
t2 = time.time()
print('Took '+str(timedelta(seconds=t2-t1)))

computing batch balanced neighbors
    finished (0:00:02.97) --> added to `.uns['neighbors']`
    'distances', weighted adjacency matrix
    'connectivities', weighted adjacency matrix
Took 0:00:03.047039


In [18]:
sc.tl.pca(adata_bbknn, svd_solver='arpack',n_comps=20)
print(adata_bbknn)
adata_bbknn.obsm['X_pca'] *= -1  # multiply by -1 to match Seurat
save_dir = os.path.join(dirname, './results/bbknn_results/')
adata_bbknn.write_csvs(save_dir, skip_data=False)
# adata_bbknn.write_h5ad(os.path.join(save_dir,'bbknn_dataset1_uc3.h5ad'))

writing '.csv' files to /data_n3/home/hoatran/python-workspace/dca_scGen/results/bbknn_results


In [8]:
# Function to plot TSNE
def plotTSNE(adata, color_group, n_pcs=20, perplexity=90, save_filename='tsne', use_repx = False):
    #adata.var_names_make_unique()
    if use_repx:
        sc.tl.tsne(adata, random_state=0, n_pcs=n_pcs, perplexity=perplexity, use_rep='X')
    else:    
        sc.tl.tsne(adata, random_state=0, n_pcs=n_pcs, perplexity=perplexity)
    sc.pl.tsne(adata, color = color_group, show=False, wspace=.3)
    save_images(save_filename)     
    
    
def plotUMAP(adata, color_group, save_filename, use_repx = False):
    
    if use_repx:
        sc.pp.neighbors(adata, use_rep='X')
    else:    
        sc.pp.neighbors(adata,n_neighbors=10, n_pcs=20)
        
    sc.tl.umap(adata)
    sc.pl.umap(adata, color = color_group, show=False)
    save_images(save_filename) 
    

In [9]:
# Visualization
# sc.tl.umap(adata_bbknn)
sc.tl.louvain(adata_bbknn)
# sc.pl.umap(adata_bbknn, color=['batch','cell_type',"louvain"])
color_group = ["batch","cell_type"]
plotUMAP(adata_bbknn, color_group, 'bbknn_umap_corrected_dataset1_uc3')
plotTSNE(adata_bbknn, color_group, 20, 90, 'bbknn_tsne_corrected_dataset1_uc3')

running Louvain clustering
    using the "louvain" package of Traag (2017)
    finished (0:00:00.07) --> found 5 clusters and added
    'louvain', the cluster labels (adata.obs, categorical)
computing neighbors
    using 'X_pca' with n_pcs = 20
    finished (0:00:00.15) --> added to `.uns['neighbors']`
    'distances', distances for each pair of neighbors
    'connectivities', weighted adjacency matrix
computing UMAP
    finished (0:00:02.18) --> added
    'X_umap', UMAP coordinates (adata.obsm)
computing tSNE
    using 'X_pca' with n_pcs = 20
    using the 'MulticoreTSNE' package by Ulyanov (2017)
    finished (0:00:03.98) --> added
    'X_tsne', tSNE coordinates (adata.obsm)


In [ ]:
# Visualization
print(adata_bbknn)
colnu = []
for i in range(adata_bbknn.obsm['X_umap'].shape[1]):
    colnu.append("UMAP"+str(i+1))
df = pd.DataFrame(adata_bbknn.obsm['X_umap'], columns=colnu, index=adata_bbknn.obs_names)
df['batch'] = pd.Series(adata_bbknn.obs['batch'], index=adata_bbknn.obs_names)
df['celltype'] = pd.Series(adata_bbknn.obs['cell_type'], index=adata_bbknn.obs_names)
df.to_csv(bdir_name+'bbknn_umap.csv')

In [ ]:
# ASW 
def write_to_csv(mat, genesname, cellsname, filename, save_dir):
    if isinstance(mat, np.ndarray):
        df = pd.DataFrame(mat, columns=genesname, index=cellsname)
    else:
        df = pd.DataFrame(mat.toarray(), columns=genesname, index=cellsname)        
    
    df.to_csv(save_dir+filename)  
    
filename = 'bbknn_pca.csv'
coln_pca = []
for i in range(adata_bbknn.obsm['X_pca'].shape[1]):
    coln_pca.append("X_pca"+str(i+1))
    

write_to_csv(adata_bbknn.obsm['X_pca'], coln_pca, adata_bbknn.obs_names,filename, save_dir)

In [10]:
# Check the changement of values before scGen and after batch correction using scGen 
import scipy.stats as st
def describe_data(adata, ax, indx):    
    desc = st.describe(adata.X[indx], axis=ax)
    print('# of observations:', desc.nobs)
    print('min: %d\nmax: %d' % desc.minmax)
    print('mean: %.1f' % desc.mean)
    
# Get one cell from batch 1, check the values before normalization and after   
print("Cell id: 10 in ",adata1.obs['batch'][10],adata1.obs_names[10])
print("Gene expression values before scanorama:")
describe_data(adata1, 0, 10) #cell id 10 in batch 1
print(adata1.X[10])
print("Cell id: 10 in ",adata_bbknn.obs['batch'][10],adata_bbknn.obs_names[10])
print("Gene expression values after scanorama:")
print(adata_bbknn.X[10])
describe_data(adata_bbknn, 0, 10) #cell id 10 in batch 1

Cell id: 10 in  Batch1 pDC_P10_S83
Gene expression values before scanorama:
# of observations: 16594
min: 0
max: 38470
mean: 60.2
[ 4.34  0.    7.49 ...  0.   98.93  0.  ]
Cell id: 10 in  Batch1 pDC_P10_S83
Gene expression values after scanorama:
[ 4.34  0.    7.49 ...  0.   98.93  0.  ]
# of observations: 16594
min: 0
max: 38470
mean: 60.2


In [11]:
# Get one cell from batch 1, check the values before normalization and after   
print("Cell id: 500 in ",adata1.obs['batch'][500],adata1.obs_names[500])
print("Gene expression values before scanorama:")
describe_data(adata1, 0, 500) #cell id 10 in batch 1
print(adata1.X[500])
print("Cell id: 500 in ",adata_bbknn.obs['batch'][500],adata_bbknn.obs_names[500])
print("Gene expression values after scanorama:")
print(adata_bbknn.X[500])
describe_data(adata_bbknn, 0, 500) #cell id 10 in batch 

Cell id: 500 in  Batch2 CD1C_P14_S78
Gene expression values before scanorama:
# of observations: 16594
min: 0
max: 135216
mean: 60.2
[0. 0. 0. ... 0. 0. 0.]
Cell id: 500 in  Batch2 CD1C_P14_S78
Gene expression values after scanorama:
[0. 0. 0. ... 0. 0. 0.]
# of observations: 16594
min: 0
max: 135216
mean: 60.2


In [12]:
# adata_bbknn.X[1:10,1:10]

In [13]:
# adata.X[1:10,1:10]

In [14]:
# Compute the silhouette coefficient score, compare the batch mixing level of filtered data and normalized data
from sklearn.metrics import silhouette_score
import seaborn as sns
sil_orig = silhouette_score(adata1.obsm.X_pca[:, :20], adata1.obs['batch'])
print(np.median(sil_orig))
# sil_true = silhouette_score(corrected_adata.obsm.X_pca[:, :2], adata_true.obs.Group)
sil_corr = silhouette_score(adata_bbknn.obsm.X_pca[:, :20], adata_bbknn.obs['batch'])
print(np.median(sil_corr))
sns.barplot(x=['Filtered', 'BBKNN-Normalized'], y=[sil_orig, sil_corr])
pl.title('Silhouette Coeff Batch BBKNN')
save_images('bbknn_silhouette_coeff_batch_label') 

0.17044699
0.17044693


In [15]:
distances, connectivities = bbknn.bbknn_pca_matrix(adata1.obsm.X_pca[:, :20], adata1.obs['batch'])

computing batch balanced neighbors
    finished (0:00:00.16) 

In [16]:
distances[1:5,1:5]

<4x4 sparse matrix of type '<class 'numpy.float64'>'
	with 0 stored elements in Compressed Sparse Row format>

In [17]:
?bbknn.bbknn